# Week 6 - Creating and Altering the Database

Need to get consent to collect data, especially personal data  
Need lookup table to alias data inputs   

## Lecture Notes: SQL Constraints and Database Alterations

### I. SQL Constraints: Ensuring Data Integrity

* **Importance:** Enforce business rules and maintain data accuracy and consistency within the database.
* **Entity Integrity:** Ensures each row in a table is uniquely identifiable.
    * **PRIMARY KEY Constraint:** Uniquely identifies each record in a table. Automatically enforces entity integrity.
        * Specified in `CREATE TABLE` using `PRIMARY KEY (column_name)`.
        * Can also be defined as a column constraint (e.g., `P_CODE VARCHAR2(10) PRIMARY KEY`).
        * Oracle automatically names constraints if not explicitly named. **Best practice:** Assign meaningful names.
* **Referential Integrity:** Maintains consistency across related tables.
    * **FOREIGN KEY Constraint:** Establishes a link between columns in two tables (child table references the parent table).
        * Specified in `CREATE TABLE` using `FOREIGN KEY (child_column) REFERENCES parent_table(parent_column)`.
        * **ON UPDATE CASCADE:** Automatically propagates changes to the primary key in the parent table to the matching foreign key values in the child table (ANSI SQL standard, *not supported by Oracle*).
        * **ON DELETE CASCADE:** Automatically deletes related rows in the child table when a row is deleted from the parent table (ANSI SQL standard). Recommended for weak entities.
        * **ON DELETE SET NULL / ON UPDATE SET NULL:** Sets the foreign key value to NULL in the child table when the corresponding primary key is updated or deleted in the parent table (ANSI SQL standard, supported by Oracle and MySQL for `SET NULL`).
        * **ON DELETE SET DEFAULT / ON UPDATE SET DEFAULT:** Sets the foreign key value to a default value in the child table (ANSI SQL standard).
        * Default behavior upon deletion of a parent record is to restrict the deletion if child records exist.
* **Other Constraints (ANSI SQL Standard):**
    * **NOT NULL:** Ensures a column cannot contain NULL values.
    * **UNIQUE:** Ensures all values in a column are distinct. Creates a unique index.
    * **DEFAULT:** Assigns a default value to a column when a new row is inserted without a specified value.
    * **CHECK:** Validates data against a specified condition when an attribute value is entered or modified.
* **Constraint Definition Locations:**
    * **Column Constraint:** Defined within the column definition. Applies only to that column.
    * **Table Constraint:** Defined using the `CONSTRAINT` keyword, usually at the end of the `CREATE TABLE` statement. Can apply to one or more columns.



### II. Creating Tables with SELECT Statements

* Allows rapid creation of a new table based on a subquery of an existing table.
* Copies attribute names, data types, and rows from the result set of the `SELECT` statement.
* **MySQL Syntax:**
    ```sql
    CREATE TABLE [IF NOT EXISTS] new_table_name AS
    SELECT column1 AS new_column1, column2 AS new_column2, ...
    FROM existing_table
    WHERE condition;
    ```
    * `IF NOT EXISTS`: Prevents overwriting an existing table and avoids error messages.
* **MS Access and SQL Server Syntax:**
    ```sql
    SELECT column1 AS new_column1, column2 AS new_column2, ...
    INTO new_table_name
    FROM existing_table
    WHERE condition;
    ```
    * MS Access prompts for deletion if the table exists.
    * SQL Server does not automatically offer to delete.
* **Important Note:** Integrity rules (primary keys, foreign keys) are **not** automatically applied to the new table. These need to be defined using `ALTER TABLE`.



### III. SQL Indexes: Enhancing Data Retrieval

* **Purpose:** Improve the efficiency of data retrieval operations (searches, sorting). Also used to enforce uniqueness.
* **Automatic Index Creation:** DBMS automatically creates a unique index when a primary key is defined.
* **`CREATE INDEX` Command:** Creates indexes on specified columns.
    ```sql
    CREATE [UNIQUE] INDEX index_name ON table_name (column1 [, column2, ...]);
    ```
    * `UNIQUE`: Creates an index that prevents duplicate values in the indexed column(s). Useful for candidate keys.
    * Composite indexes can be created on multiple columns to improve performance for queries involving those columns together.
    * Indexes are typically created on columns frequently used in `WHERE` clauses, join conditions, and `ORDER BY` clauses.
    * Descending indexes (`DESC`) can be created to optimize queries requiring reverse order.
* **`DROP INDEX` Command:** Removes an existing index.
    ```sql
    DROP INDEX index_name [ON table_name]; -- ON table_name may be required by some DBMS (e.g., MySQL)
    ```



### IV. Altering Table Structures

* **`ALTER TABLE` Command:** Used to modify the structure of an existing table.
    ```sql
    ALTER TABLE table_name
    {ADD | MODIFY | ALTER | DROP} ... ;
    ```
* **Adding a Column:**
    ```sql
    ALTER TABLE table_name
    ADD [COLUMN] column_name data_type [constraints];
    ```
    * Avoid using `NOT NULL` when adding a column to a table with existing rows, as existing rows will have NULL values in the new column.
* **Changing Column Data Type (`MODIFY` / `ALTER`):**
    ```sql
    ALTER TABLE table_name
    MODIFY column_name new_data_type; -- Or ALTER COLUMN in some DBMS
    ```
    * Restrictions apply. Some DBMS (e.g., Oracle) may not allow data type changes on non-empty columns or if it violates referential integrity.
* **Changing Column Characteristics (`MODIFY` / `ALTER`):**
    ```sql
    ALTER TABLE table_name
    MODIFY column_name new_data_type(new_size); -- Or ALTER COLUMN in some DBMS
    ```
    * DBMS may have limitations (e.g., Oracle allows increasing but not decreasing column size for non-empty columns).
* **Adding Constraints:**
    ```sql
    ALTER TABLE table_name
    ADD CONSTRAINT constraint_name constraint_definition;
    ```
    * Used to add `PRIMARY KEY`, `FOREIGN KEY`, `UNIQUE`, `NOT NULL`, and `CHECK` constraints after table creation.
    * Multiple constraints can be added in a single `ALTER TABLE` statement.
* **Dropping a Column:**
    ```sql
    ALTER TABLE table_name
    DROP [COLUMN] column_name;
    ```
    * Restrictions apply. May not be allowed if the column is part of a foreign key relationship or if it's the only column in the table.
* **Dropping Constraints:**
    ```sql
    ALTER TABLE table_name
    DROP {PRIMARY KEY | CONSTRAINT constraint_name};
    ```
    * Constraints are dropped by their name. This highlights the importance of naming constraints.



### V. Deleting a Table

* **`DROP TABLE` Command:** Removes a table and its data from the database.
    ```sql
    DROP TABLE table_name;
    ```
* **Important Consideration:** Tables cannot be dropped if they are on the "one" side of a relationship with existing foreign key constraints in other tables. You must drop tables on the "many" side first.